In [1]:
%run classifier.py
from elasticsearch import Elasticsearch
from elasticsearch import helpers
scan = helpers.scan

loading E
loading S
loading T
loading J


In [2]:
es = Elasticsearch([{'host':'localhost','port':9200}])
es

<Elasticsearch([{'host': 'localhost', 'port': 9200}])>

In [3]:
def classify(es, author, index_prefix, sublisting, all_classifier):
    classifier = None
    idx = index_prefix + '.' + sublisting
    print(idx)
    src_results = scan(es, scroll='10m',    
        query = {
            "query": {
                "query_string" : {
                    "query" : author,
                    "default_field" : "author"
                }
            }
        }, index = idx)
    
    count=0
    for src_doc in src_results:
        count = count + 1
        if count >= 100:
            break # limit to 100 comments for performance
        
        if classifier is None:
            classifier = Classifier() 
        text = src_doc['_source']['text_body']
        classifier.preprocess(text, web=False)
        all_classifier.preprocess(text, web=False)
        
    return classifier

In [97]:
import random
src_index = 'redditors'
dest_index = 'comments'
comment_sublistings = ['controversial', 'hot', 'new', 'top'] 
src_results = es.search(scroll='360m',
    body = {
    "query": {
      "function_score": {
        "query": {
          "bool": {
            "must": [
              {
                "bool": {
                  "must_not": [
                    {
                      "exists": {
                        "field": "bm"
                      }
                    }
                  ]
                }
              },
              {
                "term": {
                  "comments_indexed": {
                    "value": True
                  }
                }
              }
            ]
          }
        },
        "random_score": {
            "seed": random.randint(1,1000)
        },
        "boost_mode": "replace"
      }
    }
  }, index = src_index, size=1000)

for src_doc in src_results['hits']['hits']:
    author_doc = src_doc['_source']
    print(author_doc['name'])
    break
    controversial = None
    hot = None
    new = None
    top = None
    update = None
    
    all_classifier = Classifier()
    for sublisting in comment_sublistings:
        classifier = classify(es, author_doc['name'], dest_index, sublisting, all_classifier)
        if classifier is not None:
            if update is None:
                update = { "doc": { "bm": {} } }
            
            res = more_magic(classifier)
            res['type'] = ''.join([key[0] for key in res.keys()])
            update['doc']['bm'][sublisting] = res
    if update is not None:
        overall = more_magic(all_classifier)
        overall['type'] = ''.join([key[0] for key in overall.keys()])
        update['doc']['bm']['overall'] = overall
        es.update(index = src_index, id = src_doc['_id'], body = update)
    else:
        update = { "doc": { "bm": {} } }
        es.update(index = src_index, id = src_doc['_id'], body = update)

Kazzack
